In [1]:
import pandas as pd
import zipfile

### Dataset con el target

Seleccion de 500 muestras de ballena, 500 de delfín y 500 de beluga

In [2]:
dataset = pd.read_csv('./train.csv.zip')
dataset.head()

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392


In [4]:
len(set(dataset['species']))

30

In [3]:
dataset.columns

Index(['image', 'species', 'individual_id'], dtype='object')

In [4]:
mammal = []
other = []
for specie in dataset['species']:
    if specie[-5:] == 'whale' and not specie[-12:] == 'killer_whale':
        mammal.append('whale')
    elif specie[-7:] == 'dolphin' or specie[-7:] == 'dolpin' or specie[-12:] == 'killer_whale':
        mammal.append('dolphin')
    elif specie == 'beluga':
        mammal.append('beluga')
    else:
        other.append(specie)

print(len(dataset.index))
print(len(mammal))
print(set(other))

51033
49800
{'bottlenose_dolpin', 'globis'}


In [5]:
mammal = []
other = []
for specie in dataset['species']:
    if specie[-5:] == 'whale' and not specie[-12:] == 'killer_whale':
        mammal.append('whale')
    elif specie[-7:] == 'dolphin' or specie[-6:] == 'dolpin' or specie[-12:] == 'killer_whale':
        mammal.append('dolphin')
    elif specie == 'beluga':
        mammal.append('beluga')
    else:
        mammal.append('globis')

print(len(dataset.index))
print(len(mammal))
print(set(other))

51033
51033
set()


In [6]:
dataset['mammal group'] = mammal
dataset.head()

,image,species,individual_id,mammal group
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,whale
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,whale
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,dolphin
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,dolphin
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,whale


In [7]:
whales = dataset[dataset['mammal group'] == 'whale']
dolphins = dataset[dataset['mammal group'] == 'dolphin']
beluga = dataset[dataset['mammal group'] == 'beluga']
globis = dataset[dataset['mammal group'] == 'globis']

print('whales' , len(whales.index))
print('dolphins' , len(dolphins.index))
print('beluga' , len(beluga.index))
print('globis' , len(globis.index))

whales 21584
dolphins 21890
beluga 7443
globis 116


##### Seleccion de las 500 imagenes de cada una

In [8]:
whales_500 = whales.sample(n= 1500 , random_state= 98)
dolphins_500 = dolphins.sample(n= 1500 , random_state= 98)
beluga_500 = beluga.sample(n= 1500 , random_state= 98)


In [9]:
print('whales' , len(whales_500.index))
print('dolphins' , len(dolphins_500.index))
print('beluga' , len(beluga_500.index))

whales 1500
dolphins 1500
beluga 1500


In [10]:
dataset_final = pd.concat([whales_500 , dolphins_500 , beluga_500])
dataset_final.head()

,image,species,individual_id,mammal group
32012,a0f395c011c8c5.jpg,melon_headed_whale,1d5f22dd073f,whale
41393,d002f1c2c457cb.jpg,fin_whale,4cc267daf5e7,whale
43768,dbd8c377806229.jpg,blue_whale,9ed88b6a0ba2,whale
21383,6bcde91c2891df.jpg,gray_whale,626fc1a305f3,whale
29345,93a403e47ce37c.jpg,humpback_whale,8fdfde3b5619,whale


In [11]:
dataset_final=dataset_final.sample(frac=1).reset_index(drop=True)
dataset_final.head(10)

,image,species,individual_id,mammal group
0,dfaa26cf13a214.jpg,beluga,19b638e11443,beluga
1,6cfa1b234f2a04.jpg,killer_whale,0e461620e872,dolphin
2,440b6036e54ce8.jpg,bottlenose_dolpin,c02b7ad6faa0,dolphin
3,7fba21051447fd.jpg,beluga,babd014300b7,beluga
4,d381ef7ac5d5a3.jpg,beluga,5ac053677ed1,beluga
5,6d2a792592ad13.jpg,beluga,124534ac8131,beluga
6,1a25a16fe59bbe.jpg,bottlenose_dolphin,2ff676fd700f,dolphin
7,439580f1fc9488.jpg,white_sided_dolphin,cfacd68a2a32,dolphin
8,e3c359067831b3.jpg,bottlenose_dolphin,70cd1a7d97a7,dolphin
9,db050e682e4d7e.jpg,spinner_dolphin,b031fecf7ab2,dolphin


In [12]:
categories = []
for mammal in dataset_final['mammal group']:
    if mammal == 'dolphin':
        categories.append(0)
    elif mammal == 'whale':
        categories.append(1)
    else:
        categories.append(2)
dataset_final['categories'] = categories
dataset_final.head()

,image,species,individual_id,mammal group,categories
0,dfaa26cf13a214.jpg,beluga,19b638e11443,beluga,2
1,6cfa1b234f2a04.jpg,killer_whale,0e461620e872,dolphin,0
2,440b6036e54ce8.jpg,bottlenose_dolpin,c02b7ad6faa0,dolphin,0
3,7fba21051447fd.jpg,beluga,babd014300b7,beluga,2
4,d381ef7ac5d5a3.jpg,beluga,5ac053677ed1,beluga,2


In [13]:
dataset_final.to_csv('../model_preparation/data.csv' , sep= ';' , index= False)

### Extraccion de las imagenes correspondientes desde la carpeta zip

In [14]:
zip_path = 'C:/Users/jorar/Desktop/train_images.zip'
try:
    fh = open(zip_path , 'rb')
    z = zipfile.ZipFile(fh)
    contador = 0
    for name in z.namelist():
        if name in list(dataset_final['image']):

            outpath = "../model_preparation/cetaceous_images"
            z.extract(name, outpath)

    fh.close()
    print('Finished unzip.')
except:
    print('You need to download the original zip file from Kaggle and set the path. \nhttps://www.kaggle.com/c/happy-whale-and-dolphin')

Finished unzip.
